In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('cleaned_1km.csv')
df = pd.read_csv('cleaned_district.csv')

## This is the code book of the data

- 'building_type': The classification of a particular building.
- 'village': The region that it belongs to.
- 'district': administrative division that the Community belongs to.
- 'floor_level': The level on which a particular room or apartment is, within a building.
- 'new_lng': the longitude coordinates.
- 'new_lat': the latitude coordinates.
- 'year': time id.
- 'floor_ratio': The ratio of the floor area to the total plot area.
- 'green_ratio': The ratio of the green space to the total plot area.
- 'nego_times': The number of times a negotiation was held.
- 'lead_times': The time it takes before a deal is made.
- 'total_building': The total number of buildings in an area.
- 'total_resident': The total number of residents in an area.
- 'watching_people': The number of people watching a listing.
- 'watched_times': The number of times a listing is watched.
- 'striker_price': The initial asking price.
- 'striker_price_pers': The asking price per square foot.
- 'end_price': The final agreed price.
- 'end_price_pers': The final agreed price per square foot.
- 'area': The area of a property.
- 'nego_period': The period over which negotiations took place.
- 'bedroom': The number of bedrooms in a property.
- 'living_room': The number of living rooms in a property.
- 'kitchen': The number of kitchens in a property.
- 'toilet': The number of toilets in a property.
- 'total_floor_number': The number of floors in a building.
- 'elevator_ratio': The ratio of elevators to the total number of floors.
- 'house_age': The age of the house.
- 'income': The income of the potential buyer.
- 'number': The number of properties listed.
- 'super': referring to proximity to supermarkets (measured by number within given distance).
- 'sub': referring to proximity to subway stations.
- 'hotel': referring to proximity to hotels
- 'kind': referring to proximity to kindergartens
- 'prim': referring to primary schools.
- 'mid': referring to middle schools.
- 'shop_mall': referring to shopping mall.
- 'west_food': referring to the availability of western food nearby.
- 'park': referring to parks.
- 'museum': Distance to the nearest museum.
- 'ktv': referring to KTV and some entertainment venues.
- 'jiadian': referring to electronic shops.
- 'old': referring to old care systems.
- 'other': other real estate brokerages within 1km.
- 'other_5': other real estate brokerages within 0.5km.
- 'lianjia': lianjia's number within 1km.
- 'lianjia_5': lianjia's number within 0.5km.
- 'beke': beke's number within 1km.
- 'beke_5': beke's number within 0.5km.
- 'geometry': geometry information.
- 'light': night time lights.
- 'pop': population density.
- 'pm25': Air quality measure.
- 'region': city name.
- 'id': unique id.
- 'business_area': business area.
- 'index_right': unique index id 
- 'num': transaction number
- 'prft': lianjia's income 
- 'price' housing price

now we extract the number of transactions within the given 1km geometry to map with the community level data.